In [1]:
def limpieza(dataset):
    
    df = dataset.copy()

    df.rename(columns={'default payment next month':'default'}, inplace=True)

    df.drop(columns='ID', inplace=True)

    df = df[(df['EDUCATION'] != 0) & (df['MARRIAGE'] != 0)]

    df['EDUCATION'] = df['EDUCATION'].apply(lambda x: 4 if x > 4 else x)

    df['SEX'] = df['SEX'].astype('category')
    df['EDUCATION'] = df['EDUCATION'].astype('category')
    df['MARRIAGE'] = df['MARRIAGE'].astype('category')
    df['default'] = df['default'].astype('category')

    df.dropna(inplace=True)

    return df

In [2]:
import glob
import pandas as pd

test_file, train_file = glob.glob(f'../files/input/*')

def load_data(directory):
    df = pd.read_csv(directory)
    df = limpieza(dataset=df)
    return df

test = load_data(directory=test_file)
train = load_data(directory=train_file)

In [3]:
def division_dataset(dataset):

    df = dataset.copy()
    x  = df.drop(columns='default')
    y  = df['default']

    return x, y

x_train, y_train = division_dataset(train)
x_test, y_test = division_dataset(test)

In [4]:
y_train.value_counts()

default
0    16228
1     4725
Name: count, dtype: int64

In [5]:
# from sklearn.utils import shuffle

# class_0 = x_train[y_train == 0]
# class_1 = x_train[y_train == 1]

# class_0_sample = class_0.sample(frac=0.3, random_state=666)

# x_train_balanced = pd.concat([class_0_sample, class_1])
# y_train_balanced = pd.concat([
#     pd.Series([0] * len(class_0_sample), index=class_0_sample.index),
#     pd.Series([1] * len(class_1), index=class_1.index)
# ])

# x_train_balanced, y_train_balanced = shuffle(x_train_balanced, y_train_balanced, random_state=666)

# y_train_balanced.value_counts()

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPClassifier

In [7]:
categorical_columns = x_train.select_dtypes(include='category').columns.to_list()

column_transformer = ColumnTransformer(
    [
        #('categories', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse_output=False), categorical_columns),
        ('categories', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns),
    ],
    remainder=StandardScaler()
)

pipe = Pipeline(steps=
    [
        ('preprocessor', column_transformer),
        ('dimensionality_reducter', PCA()),
        ('k_best_selector', SelectKBest(score_func=f_classif, k='all')),
        ('classifier', MLPClassifier(solver='adam', early_stopping=True, random_state=17, learning_rate='adaptive', max_iter=1000))
    ]
)

pipe.fit(x_train, y_train)

c:\Users\arica\OneDrive\Analìtica predictiva\2024-2-LAB-04-prediccion-del-default-usando-mlp-Felipe-Arikpa\.venv\lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('categories',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['SEX', 'EDUCATION',
                                                   'MARRIAGE'])])),
                ('dimensionality_reducter', PCA()),
                ('k_best_selector', SelectKBest(k='all')),
                ('classifier',
                 MLPClassifier(early_stopping=True, learning_rate='adaptive',
                               max_iter=1000, random_state=17))])

In [8]:
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, f1_score

y_pred_train = pipe.predict(x_train)
y_pred_test = pipe.predict(x_test)

print('balanced accuracy train:', balanced_accuracy_score(y_train, y_pred_train))
print('balanced accuracy test:', balanced_accuracy_score(y_test, y_pred_test))
print('precision score train:', precision_score(y_train, y_pred_train))
print('precision score test:', precision_score(y_test, y_pred_test))
print('recall score train:', recall_score(y_train, y_pred_train))
print('recall score test:', recall_score(y_test, y_pred_test))
print('f1 score train:', f1_score(y_train, y_pred_train))
print('f1 score test:', f1_score(y_test, y_pred_test))
print()
print(pipe.get_params())

balanced accuracy train: 0.6410791198959797
balanced accuracy test: 0.6543804758915753
precision score train: 0.6807760141093474
precision score test: 0.677710843373494
recall score train: 0.32677248677248677
recall score test: 0.35414480587618047
f1 score train: 0.4415844415844416
f1 score test: 0.4651964162646451

{'memory': None, 'steps': [('preprocessor', ColumnTransformer(remainder=StandardScaler(),
                  transformers=[('categories',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['SEX', 'EDUCATION', 'MARRIAGE'])])), ('dimensionality_reducter', PCA()), ('k_best_selector', SelectKBest(k='all')), ('classifier', MLPClassifier(early_stopping=True, learning_rate='adaptive', max_iter=1000,
              random_state=17))], 'transform_input': None, 'verbose': False, 'preprocessor': ColumnTransformer(remainder=StandardScaler(),
                  transf

In [9]:
from sklearn.metrics import confusion_matrix

def matriz(yt, yp):
    matrix = confusion_matrix(y_true=yt, y_pred=yp)
    return matrix

matrix_train = confusion_matrix_df = pd.DataFrame(
    matriz(yt=y_train, yp=pipe.predict(x_train)),
    index=["N (Clase Real Negativa)", "P (Clase Real Positiva)"],
    columns=["PN (Predicción Negativa)", "PP (Predicción Positiva)"]
)

matrix_test = confusion_matrix_df = pd.DataFrame(
    matriz(yt=y_test, yp=pipe.predict(x_test)),
    index=["N (Clase Real Negativa)", "P (Clase Real Positiva)"],
    columns=["PN (Predicción Negativa)", "PP (Predicción Positiva)"]
)

display(
    matrix_train,
    matrix_test
)

,PN (Predicción Negativa),PP (Predicción Positiva)
N (Clase Real Negativa),15504,724
P (Clase Real Positiva),3181,1544


,PN (Predicción Negativa),PP (Predicción Positiva)
N (Clase Real Negativa),6752,321
P (Clase Real Positiva),1231,675


In [10]:
columns_transformer = pipe.named_steps['preprocessor']

x_transformed = columns_transformer.transform(x_train)
x_transformed = pd.DataFrame(x_transformed, columns=columns_transformer.get_feature_names_out())

print('columnas dataset original:', len(train.columns))
print('columnas dataset transformado:', len(x_transformed.columns))

columnas dataset original: 24
columnas dataset transformado: 29


In [11]:
pca = pipe.named_steps['dimensionality_reducter']
x_reduced = pca.transform(x_transformed)

varianza_explicada = pca.explained_variance_ratio_ * 100
varianza_explicada = pd.DataFrame(
    {
        'Componente': pca.get_feature_names_out(),
        'Varianza Explicada (%)': varianza_explicada
    }
)

varianza_explicada['Varianza Acumulada (%)'] = varianza_explicada['Varianza Explicada (%)'].cumsum()
varianza_explicada['Varianza Explicada (%)'] = varianza_explicada['Varianza Explicada (%)'].apply(lambda x:f'{x:.2f}')

varianza_explicada  

c:\Users\arica\OneDrive\Analìtica predictiva\2024-2-LAB-04-prediccion-del-default-usando-mlp-Felipe-Arikpa\.venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(


,Componente,Varianza Explicada (%),Varianza Acumulada (%)
0,pca0,30.41,30.413653
1,pca1,18.74,49.155896
2,pca2,7.11,56.269750
3,pca3,5.39,61.663861
4,pca4,4.31,65.974764
5,pca5,4.06,70.039720
6,pca6,3.93,73.969865
7,pca7,3.56,77.525024
8,pca8,3.45,80.973230
9,pca9,3.28,84.255162


In [12]:
kbest = pipe.named_steps['k_best_selector'] 

feature_names = pca.get_feature_names_out()

anova = pd.DataFrame(
    {
        'feature':feature_names,
        'F-Score':kbest.scores_,
        'P-Value':[f'{p:.6%}' for p in kbest.pvalues_],
    }
).sort_values(by='F-Score', ascending=False).reset_index(drop=True)

anova

,feature,F-Score,P-Value
0,pca1,1831.593327,0.000000%
1,pca9,313.936505,0.000000%
2,pca0,148.216456,0.000000%
3,pca10,139.823744,0.000000%
4,pca12,132.662209,0.000000%
5,pca28,85.019088,0.000000%
6,pca3,82.207523,0.000000%
7,pca26,24.178490,0.000088%
8,pca24,21.637731,0.000331%
9,pca16,21.544163,0.000348%


In [13]:
from sklearn.inspection import permutation_importance

permutation = permutation_importance(
    estimator=pipe,
    X=x_test,
    y=y_test,
    scoring='balanced_accuracy',
    n_jobs=-1,
    random_state=17,
    n_repeats=10
)

importances = permutation.importances_mean

importance_features = list(zip(pipe[:-1].get_feature_names_out(), importances))
importance_features = sorted(importance_features, key=lambda x: x[1], reverse=True)

for feature, importance in importance_features:
    print(f'{feature}: {importance:.6}')

pca5: 0.116644
pca0: 0.0126174
pca9: 0.0110193
pca6: 0.0106575
pca7: 0.0102566
pca14: 0.00576157
pca15: 0.00441756
pca11: 0.00433567
pca13: 0.00402252
pca12: 0.00370116
pca10: 0.00359368
pca3: 0.00220749
pca16: 0.00174203
pca21: 0.00162517
pca22: 0.00156768
pca19: 0.000813055
pca18: 0.00052543
pca17: 0.000493234
pca8: 9.71951e-05
pca1: -0.000139506
pca20: -0.000868502
pca2: -0.000987572
pca4: -0.00271077


In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import numpy as np

# def custom_metric(yt, yp):
#     precision = precision_score(y_true=yt, y_pred=yp)
#     balanced = balanced_accuracy_score(y_true=yt, y_pred=yp)
#     if precision >= 0.691 and balanced >= 0.661:
#         return precision + balanced
#     else:
#         return balanced

# scoring = make_scorer(custom_metric)


# param_grid = {
#     'classifier__hidden_layer_sizes': [
#           (i, j, k, l)
#           for i in range(108, 117)]
#           for j in range(80, 89)]
#           for k in range(52, 61)]
#           for l in range(24, 33)]
# }

#Mejores resultados hasta ahora:
#[(308, 280, 252, 224, 196, 168, 140, 112, 84, 56, 28)]
#[(112, 84, 56, 28)]
#(112, 84, 56, 28, 7, 7)
#[(308, 280, 252, 224, 196, 168, 140, 112, 84, 56, 28, 5, 4, 3)]

param_grid = {
    'classifier__hidden_layer_sizes': [(112, 84, 56, 28), ((112, 84, 56, 28, 7, 7))],
    #'k_best_selector__k': list(range(27, 29)),
    #'classifier__alpha': [0, 0.001, 0.01, 0.1, 1, 2, 3],
    #'classifier__alpha': np.arange(0.0001, 0.00011, 0.00001),
    'classifier__random_state': [None, None, 17],
    'classifier__batch_size': ['auto', 64, 32]
}

model = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=10,
    scoring='balanced_accuracy',
    n_jobs=-1,
    refit=True,
    verbose=1,
)

model.fit(x_train, y_train)

Fitting 10 folds for each of 18 candidates, totalling 180 fits


c:\Users\arica\OneDrive\Analìtica predictiva\2024-2-LAB-04-prediccion-del-default-usando-mlp-Felipe-Arikpa\.venv\lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('categories',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE'])])),
                                       ('dimensionality_reducter', PCA()),
                                       ('k_best_selector',
                                        SelectKBest(k='all')),
                                       ('classifier',
                                        MLPClassifier(early_stopping=True,
                                                      learning_rate='adaptive',
                                                      max_iter=1000,
                                                      random_state=17))]),
             n_jobs=-1,
             param_grid={'classifier__batch_size': ['auto', 64, 32],
                         'classifier__hidden_layer_sizes': [(112, 84, 56, 28),
                                                            (112, 84, 56, 28, 7,
                                                             7)],
                         'classifier__random_state': [None, None, 17]},
             scoring='balanced_accuracy', verbose=1)

In [15]:
y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)

print('mejores parámetros encontrados:', model.best_params_)
print()
print('balanced accuracy train:', balanced_accuracy_score(y_train, y_pred_train))
print('balanced accuracy test:', balanced_accuracy_score(y_test, y_pred_test))
print('precision score train:', precision_score(y_train, y_pred_train))
print('precision score test:', precision_score(y_test, y_pred_test))
print('recall score train:', recall_score(y_train, y_pred_train))
print('recall score test:', recall_score(y_test, y_pred_test))
print('f1 score train:', f1_score(y_train, y_pred_train))
print('f1 score test:', f1_score(y_test, y_pred_test))

mejores parámetros encontrados: {'classifier__batch_size': 'auto', 'classifier__hidden_layer_sizes': (112, 84, 56, 28, 7, 7), 'classifier__random_state': 17}

balanced accuracy train: 0.6668975433407279
balanced accuracy test: 0.6748277853101126
precision score train: 0.6804469273743017
precision score test: 0.6652286453839517
recall score train: 0.38666666666666666
recall score test: 0.40451206715634835
f1 score train: 0.4931174089068826
f1 score test: 0.5030995106035889


In [16]:
results = pd.DataFrame(model.cv_results_)

results = results[
    [
        #'param_k_best_selector__k',
        #'param_classifier__alpha',
        'param_classifier__hidden_layer_sizes',
        'param_classifier__random_state',
        'param_classifier__batch_size',
        'mean_test_score'
    ]
].sort_values('mean_test_score', ascending=False).reset_index(drop=True)

results.head(50)

,param_classifier__hidden_layer_sizes,param_classifier__random_state,param_classifier__batch_size,mean_test_score
0,"(112, 84, 56, 28, 7, 7)",17,auto,0.664107
1,"(112, 84, 56, 28, 7, 7)",None,auto,0.662791
2,"(112, 84, 56, 28)",17,auto,0.661473
3,"(112, 84, 56, 28, 7, 7)",17,32,0.661445
4,"(112, 84, 56, 28)",None,auto,0.661318
5,"(112, 84, 56, 28, 7, 7)",None,auto,0.660977
6,"(112, 84, 56, 28, 7, 7)",None,64,0.660046
7,"(112, 84, 56, 28, 7, 7)",17,64,0.657448
8,"(112, 84, 56, 28)",17,32,0.657055
9,"(112, 84, 56, 28)",None,auto,0.656360


In [17]:
# from sklearn.model_selection import TunedThresholdClassifierCV

# turned_model = TunedThresholdClassifierCV(
#     estimator=model.best_estimator_,
#     scoring=scoring,
#     n_jobs=-1,
#     random_state=666,
#     cv=7,
#     refit=True
# )

# turned_model.fit(x_train, y_train)

# print('mejor umbral de decisión:', turned_model.best_threshold_)
# print('mejor score:', turned_model.best_score_)

In [18]:
# y_pred_train_turned = turned_model.predict(x_train)
# y_pred_test_turned = turned_model.predict(x_test)

# print('balanced accuracy train:', balanced_accuracy_score(y_train, y_pred_train_turned))
# print('balanced accuracy test:', balanced_accuracy_score(y_test, y_pred_test_turned))
# print('precision score train:', precision_score(y_train, y_pred_train_turned))
# print('precision score test:', precision_score(y_test, y_pred_test_turned))
# print('recall score train:', recall_score(y_train, y_pred_train_turned))
# print('recall score test:', recall_score(y_test, y_pred_test_turned))
# print('f1 score train:', f1_score(y_train, y_pred_train_turned))
# print('f1 score test:', f1_score(y_test, y_pred_test_turned))

In [19]:
# from tqdm import tqdm

# probabilities = model.best_estimator_.predict_proba(x_test)[:,1]

# umbrales_validos = []

# for threshold in tqdm(np.arange(turned_model.best_threshold_ - 0.1, turned_model.best_threshold_ + 0.1, 0.001), desc='Calculando umbrales'):
#     predicciones = (probabilities >= threshold).astype(int)
    
#     precision = precision_score(y_test, predicciones, zero_division=0)
#     balanced_accuracy = balanced_accuracy_score(y_test, predicciones)
#     combined_score = (precision * 0.4) + (balanced_accuracy * 0.6)
    
#     if precision >= 0.673 and balanced_accuracy >= 0.661:
#         umbrales_validos.append(
#             {
#                 'threshold': threshold,
#                 'precision': precision,
#                 'balanced_accuracy': balanced_accuracy,
#                 'combined_score': combined_score
#             }
#         )

# if umbrales_validos:
#     mejor_resultado = sorted(umbrales_validos, key=lambda x: x['combined_score'], reverse=True)[0]
#     print(mejor_resultado)
# else:
#     print('No se encontraron umbrales válidos que cumplan las condiciones.')

In [20]:
# if umbrales_validos:
#     best_threshold = mejor_resultado['threshold']
#     probabilities = model.predict_proba(x_test)[:,1]
#     threshold = (probabilities >= best_threshold).astype(int)

In [21]:
import pickle
import gzip
import os

os.makedirs('../files/models', exist_ok=True)

with gzip.open('../files/models/model.pkl.gz', 'wb') as file:
    pickle.dump(model, file)

In [22]:
import json

def calculate_metrics(modelo, x, y, tipo):

    y_pred = modelo.predict(x)
    
    metrics = {
        'type': 'metrics',
        'dataset':tipo,
        'precision':precision_score(y_pred=y_pred, y_true=y),
        'balanced_accuracy':balanced_accuracy_score(y_pred=y_pred, y_true=y),
        'recall':recall_score(y_pred=y_pred, y_true=y),
        'f1_score':f1_score(y_pred=y_pred, y_true=y)
    }
    return metrics

train_metrics = calculate_metrics(modelo=model, x=x_train, y=y_train, tipo='train')
test_metrics = calculate_metrics(modelo=model, x=x_test, y=y_test, tipo='test')

metricas = [train_metrics, test_metrics]

os.makedirs('../files/output', exist_ok=True)

with open('../files/output/metrics.json', 'w') as file:
    for metrica in metricas:
        file.write(json.dumps(metrica)+'\n')

In [23]:
from sklearn.metrics import confusion_matrix

def create_mc(modelo, x, y, tipo):

    y_pred = modelo.predict(x)

    matrix = confusion_matrix(y_true=y, y_pred=y_pred)

    dictionary = {
        'type':'cm_matrix',
        'dataset':tipo,
        'true_0':{
            'predicted_0':int(matrix[0, 0]),
            'predicted_1':int(matrix[0, 1])
        },
        'true_1':{
            'predicted_0':int(matrix[0, 1]),
            'predicted_1':int(matrix[1, 1])
        }
    }
    return dictionary

train_matrix = create_mc(modelo=model, x=x_train, y=y_train, tipo='train')
test_matrix = create_mc(modelo=model, x=x_test, y=y_test, tipo='test')

metricas_2 = [train_matrix, test_matrix]

with open('../files/output/metrics.json', 'a') as file:
    for metrica in metricas_2:
        file.write(json.dumps(metrica)+'\n')